In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
os.chdir('/Users/chrissoria/Documents/Research/BICS_Political_Polarization/')

In [3]:
all_waves = pd.read_csv('/Users/chrissoria/Documents/Research/BICS/data/df_combined.csv', low_memory=False)

For this preliminary analysis, I will only use wave 2,4,6. \
Each wave's data was collected on 2020-06-17 - 2020-06-23, 2020-11-29 - 2020-12-16, 2021-05-12 - 2021-05-25.

In [4]:
all_waves['wave_range'].unique()

array(['2020-06-17 - 2020-06-23', '2020-11-29 - 2020-12-16',
       '2021-05-12 - 2021-05-25'], dtype=object)

In [5]:
col_types = {
    'ZIP': str,
    'CD':str,
    'COUNTY_FIPS':str,
    'STATE_FIPS':str,
    'CONGRESSPERSON_PARTY': str
}

ZIP_FEATURES = pd.read_csv('/Users/chrissoria/Documents/Research/BICS_Political_Polarization/data/ZIP_Features.csv',
                          dtype=col_types)

In [6]:
col_types = {
    'resp_zip': str,
    'political_party': str
}

all_waves = pd.read_csv('/Users/chrissoria/Documents/Research/BICS/data/df_combined.csv', 
                        low_memory=False, dtype=col_types)

all_waves = all_waves.rename(columns={
    'resp_zip': 'ZIP'
})

columns_to_keep = ['ResponseId','StartDate','resp_yob','resp_sex','resp_hispanic','resp_race_1','resp_race_2',
                  'resp_race_3','resp_race_4','resp_race_5','resp_race_6','resp_educ','resp_nativity','ZIP',
                  'resp_hhsize','resp_employ','resp_occupation','lefthome_num','num_cc_nonhh',
                  'lefthome_where_1','lefthome_where_2','lefthome_where_3','lefthome_where_4','lefthome_where_10',
                  'lefthome_where_8','lefthome_where_9','lefthome_where_5','lefthome_where_11','lefthome_where_6',
                  'lefthome_where_7','inet_freq','socmedia_use','covid19_familiar','covid19_concern',
                  'covid19_f2fchange','covid19_reduceOK','policy_sip','age','hhi','political_party','political_view',
                  'industry','health_insurance','interview_date','wave','agecat','city','covid19_vax','covid19_whynot_vax']
all_waves = all_waves[columns_to_keep]

In [7]:
BICS_ZIP_FEATURES = pd.merge(all_waves, ZIP_FEATURES, on = "ZIP", how = "left")
BICS_ZIP_FEATURES

,ResponseId,StartDate,resp_yob,resp_sex,resp_hispanic,resp_race_1,resp_race_2,resp_race_3,resp_race_4,resp_race_5,...,COUNTY_COV_DEATHS_2020_12_09,COUNTY_COV_DEATHS_2021_05_19,STATE_FIPS,POPESTIMATE2020,DEATHS2020,POPESTIMATE2021,DEATHS2021,COV_County_CMR_2020_06_20,COV_County_CMR_2020_12_09,COV_County_CMR_2021_05_19
0,R_10ARlAAmJ8eGNJb,2020-06-18 21:18:16,1996,Female,No,White,NaN,NaN,NaN,NaN,...,475.0,735.0,08,655112.0,1258.0,654900.0,5228.0,0.517469,0.725067,1.122309
1,R_10ARlAAmJ8eGNJb,2020-06-18 21:18:16,1996,Female,No,White,NaN,NaN,NaN,NaN,...,475.0,735.0,08,655112.0,1258.0,654900.0,5228.0,0.517469,0.725067,1.122309
2,R_yrmrreDsETFkB3j,2020-06-18 21:18:03,1999,Female,No,NaN,NaN,NaN,Asian,NaN,...,68.0,153.0,37,87546.0,323.0,87611.0,1237.0,0.194184,0.776735,1.746356
3,R_1Cg41joJbGmfLZY,2020-06-18 21:17:47,1957,Female,No,White,NaN,NaN,NaN,NaN,...,34.0,51.0,37,69060.0,200.0,71703.0,800.0,0.304083,0.492326,0.711267
4,R_1Cg41joJbGmfLZY,2020-06-18 21:17:47,1957,Female,No,White,NaN,NaN,NaN,NaN,...,34.0,51.0,37,69060.0,200.0,71703.0,800.0,0.304083,0.492326,0.711267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16215,R_3G0RIc556HS65Y5,2021-05-26 14:25:26,1985,Male,No,White,Black or African American,American Indian or Alaska Native,NaN,NaN,...,509.0,1064.0,13,766209.0,1265.0,766802.0,5819.0,0.298874,0.664310,1.387581
16216,R_2hL2c0J6jveA1wd,2021-05-26 17:48:09,1982,Male,No,NaN,NaN,NaN,Asian,NaN,...,1336.0,2218.0,25,861461.0,2094.0,862029.0,8579.0,1.041254,1.550854,2.572999
16217,R_1q21nl5RsYQJxJu,2021-05-26 17:59:11,1967,Male,No,NaN,Black or African American,NaN,NaN,NaN,...,1200.0,1780.0,25,726025.0,1664.0,724505.0,7135.0,1.250646,1.652836,2.456850
16218,R_1q21nl5RsYQJxJu,2021-05-26 17:59:11,1967,Male,No,NaN,Black or African American,NaN,NaN,NaN,...,1200.0,1780.0,25,726025.0,1664.0,724505.0,7135.0,1.250646,1.652836,2.456850


In [8]:
BICS_ZIP_FEATURES['political_party_to_CD'] = BICS_ZIP_FEATURES.apply(
    lambda row: str(row['political_party']) + " in " + str(row['CONGRESSPERSON_PARTY']) + " led CD", axis=1
)

# Print the updated DataFrame
print(BICS_ZIP_FEATURES[['political_party', 'CONGRESSPERSON_PARTY', 'political_party_to_CD']])


            political_party CONGRESSPERSON_PARTY  \
0      Prefer not to answer           Democratic   
1      Prefer not to answer           Democratic   
2               Independent           Republican   
3               Independent           Republican   
4               Independent           Democratic   
...                     ...                  ...   
16215           Independent           Republican   
16216  Prefer not to answer           Democratic   
16217              Democrat           Democratic   
16218              Democrat           Democratic   
16219            Republican           Republican   

                           political_party_to_CD  
0      Prefer not to answer in Democratic led CD  
1      Prefer not to answer in Democratic led CD  
2               Independent in Republican led CD  
3               Independent in Republican led CD  
4               Independent in Democratic led CD  
...                                          ...  
16215           In

In [9]:
BICS_ZIP_FEATURES.to_csv('data/BICS_ZIP_Features.csv',index=False)